In [1]:
import pandas as pd 
import os

In [2]:
os.listdir()

['train',
 'EDA.ipynb',
 'train.csv',
 'sample_submission.csv',
 '.ipynb_checkpoints',
 'osic-pulmonary-fibrosis-progression.zip',
 'test',
 'test.csv']

In [3]:
df=pd.read_csv("train.csv")

In [4]:
df.head(29)

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker
5,ID00007637202177411956430,17,2101,52.868646,79,Male,Ex-smoker
6,ID00007637202177411956430,29,2000,50.327126,79,Male,Ex-smoker
7,ID00007637202177411956430,41,2064,51.937594,79,Male,Ex-smoker
8,ID00007637202177411956430,57,2057,51.761449,79,Male,Ex-smoker
9,ID00009637202177434476278,8,3660,85.282878,69,Male,Ex-smoker


In [5]:
number_of_patients=len(df.Patient.unique())

In [6]:
round(len(df)/len(df.Patient.unique()))

9

In [7]:
import pydicom

In [8]:
image_paths=[]
from os.path import join,getsize
for path,dirname,fname in os.walk(top="train",topdown=True):
    for filename in fname:
        image_paths.append(os.path.join(path,filename))
        

In [9]:
image_paths[0]

'train/ID00035637202182204917484/64.dcm'

In [10]:
import pydicom

In [11]:
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
import tensorflow_io as tfio 
import tqdm


In [ ]:
def decode_image(image_path):
    image_bytes = tf.io.read_file(image_path)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    image=np.squeeze(image.numpy())
    return image

In [3]:
#TO_GENERATE_TIMELAPSE_CTSCANS

def choose_patient(ID):
    images_x=[]
    for i in image_paths:
        name=str(i)
        if str(ID) in name:
            images_x.append(i)
    images_x.sort()
    return imgaes_x

def generate_patient_movie(images):
    for x,i in enumerate(images_x):
        image=decode_image(i)   
        fname=str(x)+".png"
        plt.imsave(fname,image,cmap='twilight_shifted')    

def save():
    os.system("ffmpeg -r 1 -i img%01d.png -vcodec mpeg4 -y movie.mp4")
 

In [ ]:
plt.imshow(z,cmap='twilight_shifted',alpha=1)